In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../bin/data.csv')

df.head(10)

,fecha_extracción,provincia,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,carroceria,num_puertas,num_plazas,peso,combustible,potencia_cv,cilindrada,tipo_cambio,num_marchas,largo,ancho,alto,capacidad_maletero,batalla,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,cilindros,sobrealimentacion,traccion,potencia_kw,par,velocidad_max,aceleracion
0,2024-11-04 16:50:01.106559,Las Palmas,Detalles MERCEDES-BENZ EQA (2021),Matriculado: 01/2021,28000,GT CARS PREMIUM LAS PALMAS,Profesional,0 EMISIONES,12 meses,35900,no disponible,"4x4, SUV o pickup",5 Puertas,5,NaN,Eléctrico,218,NaN,Automático,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-04 16:50:01.106559,Madrid,Detalles RENAULT Laguna (2001),Matriculado: 09/2001,246000,AUTOS VALDEL,Profesional,B,3 meses,2190,21300,CarroceríaBerlina,Número de puertas5,Número de plazas5,Peso/Masa max. autorizado (kg)1.865,Combustible Gasolina,Potencia (CV)123,Cilindrada (cm3) 1.783,Tipo de transmisión Manual,Número de marchas5,4576 mm Largo,1772 mm Ancho,1429 mm Alto,Capacidad del maletero (l)0/0,Batalla (mm)\r\n2.740,"Consumo medio 7,50 litros/100 km","Consumo en carretera 6,10 litros/100 km","Consumo urbano 10,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 0,00 litros",Nº de cilindros 4 cilindros,Sobrealimentación -,TracciónTracción delantera,Potencia (KW)89,Par (Nm)168,Velocidad máxima (km/h)-,"Aceleración 0-100 km/h (s)10,70"
2,2024-11-04 16:50:01.106559,Ciudad Real,Detalles KGM / SSANGYONG Tivoli (2019),Matriculado: 10/2019,77440,AUTOS JIMECAR,Profesional,C,12 meses,15990,21063,CarroceríaTodo Terreno,Número de puertas5,Número de plazas5,Peso/Masa max. autorizado (kg)1.880,Combustible Gasolina,Potencia (CV)163,Cilindrada (cm3) 1.497,Tipo de transmisión Automática secuencial,Número de marchas6,4225 mm Largo,1810 mm Ancho,1613 mm Alto,Capacidad del maletero (l)427/1.115,Batalla (mm)\r\n2.600,"Consumo medio 7,50 litros/100 km","Consumo en carretera 6,40 litros/100 km","Consumo urbano 9,30 litros/100 km",CO2 (g/km) 160 g/km,"Depósito de combustible (l) 50,00 litros",Nº de cilindros 4 cilindros,Sobrealimentación Turbo,TracciónTracción delantera,Potencia (KW)120,Par (Nm)260,Velocidad máxima (km/h)175,"Aceleración 0-100 km/h (s)0,00"
3,2024-11-04 16:50:01.106559,Madrid,Detalles FORD Puma (2023),Matriculado: 06/2023,29011,ROMADRID CAR SLU,Profesional,ECO,19 meses,18990,26293,CarroceríaTodo Terreno,Número de puertas5,Número de plazas5,Peso/Masa max. autorizado (kg)1.760,Combustible Gasolina,Potencia (CV)125,Cilindrada (cm3) 999,Tipo de transmisión Manual,Número de marchas6,4186 mm Largo,1805 mm Ancho,1537 mm Alto,Capacidad del maletero (l)402/1.216,Batalla (mm)\r\n2.588,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 42,00 litros",Nº de cilindros 3 cilindros,Sobrealimentación Turbo,TracciónTracción delantera,Potencia (KW)92,Par (Nm)210,Velocidad máxima (km/h)191,"Aceleración 0-100 km/h (s)9,80"
4,2024-11-04 16:50:01.106559,Vizcaya,Detalles CITROEN C4 (2024),Matriculado: 09/2024,10,"PEUGEOT KAREALDE, Stock Km0",Profesional,C,36 meses,20760,26750,CarroceríaBerlina,Número de puertas5,Número de plazas5,Peso/Masa max. autorizado (kg)1.735,Combustible Gasolina,Potencia (CV)130,Cilindrada (cm3) 1.199,Tipo de transmisión Manual,Número de marchas6,4360 mm Largo,1834 mm Ancho,1520 mm Alto,Capacidad del maletero (l)380/408,Batalla (mm)\r\n2.670,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 60,00 litros",Nº de cilindros 3 cilindros,Sobrealimentación -,TracciónTracción delantera,Potencia (KW)96,Par (Nm)230,Velocidad máxima (km/h)210,"Aceleración 0-100 km/h (s)10,80"
5,2024-11-04 16:50:01.106559,Madrid,Detalles MERCEDES-BENZ Clase A (2021),Matriculado

In [4]:
df.columns

Index(['fecha_extracción', 'provincia', 'marca_modelo', 'fecha_matriculacion',
       'kilometraje', 'nombre_vendedor', 'tipo_vendedor',
       'distintivo_ambiental', 'garantia', 'precio_contado', 'precio_nuevo',
       'carroceria', 'num_puertas', 'num_plazas', 'peso', 'combustible',
       'potencia_cv', 'cilindrada', 'tipo_cambio', 'num_marchas', 'largo',
       'ancho', 'alto', 'capacidad_maletero', 'batalla', 'consumo_medio',
       'consumo_carretera', 'consumo_urbano', 'CO2', 'deposito', 'cilindros',
       'sobrealimentacion', 'traccion', 'potencia_kw', 'par', 'velocidad_max',
       'aceleracion'],
      dtype='object')

In [5]:
df['kilometraje'].dtype == 'int64'

True

In [6]:
df.shape

(30, 37)

In [7]:
nan_percentage = df.isna().mean()*100
print(nan_percentage)

fecha_extracción        0.000000
provincia               0.000000
marca_modelo            0.000000
fecha_matriculacion     0.000000
kilometraje             0.000000
nombre_vendedor         0.000000
tipo_vendedor           0.000000
distintivo_ambiental    0.000000
garantia                0.000000
precio_contado          0.000000
precio_nuevo            0.000000
carroceria              0.000000
num_puertas             0.000000
num_plazas              0.000000
peso                    3.333333
combustible             0.000000
potencia_cv             0.000000
cilindrada              3.333333
tipo_cambio             0.000000
num_marchas             3.333333
largo                   3.333333
ancho                   3.333333
alto                    3.333333
capacidad_maletero      3.333333
batalla                 3.333333
consumo_medio           3.333333
consumo_carretera       3.333333
consumo_urbano          3.333333
CO2                     3.333333
deposito                3.333333
cilindros 

In [8]:
df = df.dropna(subset=['precio_contado'])

In [9]:
nan_percentage = df.isna().mean()*100
print(nan_percentage)

fecha_extracción        0.000000
provincia               0.000000
marca_modelo            0.000000
fecha_matriculacion     0.000000
kilometraje             0.000000
nombre_vendedor         0.000000
tipo_vendedor           0.000000
distintivo_ambiental    0.000000
garantia                0.000000
precio_contado          0.000000
precio_nuevo            0.000000
carroceria              0.000000
num_puertas             0.000000
num_plazas              0.000000
peso                    3.333333
combustible             0.000000
potencia_cv             0.000000
cilindrada              3.333333
tipo_cambio             0.000000
num_marchas             3.333333
largo                   3.333333
ancho                   3.333333
alto                    3.333333
capacidad_maletero      3.333333
batalla                 3.333333
consumo_medio           3.333333
consumo_carretera       3.333333
consumo_urbano          3.333333
CO2                     3.333333
deposito                3.333333
cilindros 

In [10]:
df.shape

(30, 37)

In [11]:
df['tipo_cambio'].unique()

array(['Automático', 'Tipo de transmisión Manual',
       'Tipo de transmisión Automática secuencial'], dtype=object)

In [12]:
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Automático' if 'Automátic' in x else 'Manual')

In [13]:
df['carroceria'].unique()

array(['4x4, SUV o pickup', 'CarroceríaBerlina', 'CarroceríaTodo Terreno',
       'CarroceríaMonovolumen', 'CarroceríaCoupe', 'CarroceríaRoadster',
       'CarroceríaStationwagon'], dtype=object)

In [14]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

In [15]:
df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [16]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

In [17]:
df = df.drop(columns=['fecha_matriculacion'])

In [18]:
df['ano_matriculacion']=df['ano_matriculacion'].astype('int64')

In [19]:
df['mes_matriculacion']=df['mes_matriculacion'].astype('int64')

In [20]:
df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [21]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [22]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

In [23]:
df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

In [24]:
df["largo"] = df["largo"].str.split(' ').str[0].replace("-", np.nan).astype(float)

In [25]:
df["ancho"] = df["ancho"].str.split(' ').str[0].replace("-", np.nan).astype(float)

In [26]:
df["alto"] = df["alto"].str.split(' ').str[0].replace("-", np.nan).astype(float)

In [27]:
df["capacidad_maletero"] = df["capacidad_maletero"].str.split(")").str[1].str.split("/").str[0].astype(float)

In [28]:
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype(float)

In [29]:
df["batalla"] = df["batalla"].str.extract(r'(\d+\.\d+)').replace(".","").astype(float)

In [30]:
df["peso"] = df["peso"].str.extract(r'(\d+\.\d+)').replace(".","").astype(float)

In [31]:
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype(int)

In [32]:
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

In [33]:
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

In [34]:
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

In [35]:
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)

In [36]:
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

In [37]:
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype(float)

In [38]:
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)').replace(',', '.', regex=True).astype(float)

In [39]:
df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype(float)

In [40]:
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype(float)

In [41]:
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype(float)

In [42]:
df["par"] = df["par"].str.extract(r'(\d+)').astype(float)

In [43]:
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype(float)

In [44]:
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)

In [45]:
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '') if pd.notna(x) else x)

In [47]:
marca_listado = pd.read_csv('../bin/listado_marcas.csv')

#La lista de marcas no está en mayúsculas pero en "data" sí. Hay que convertir todo a mayúsculas
lista_marcas = [marca.upper() for marca in marca_listado['nombre_marca'].tolist()]

df['marca_modelo'] = df['marca_modelo'].str.replace("Detalles ", "").str.strip().str.upper()

#En la línea de abajo, "next" busca el siguiente elemento de la lista para comprobar que coincide
df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: x['marca_modelo'].replace(x['marca'], "").strip(" ()") if x['marca'] else x['marca_modelo'], axis=1)


TypeError: replace() argument 1 must be str, not float

In [ ]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion'])

In [ ]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'carroceria': 'id_carroceria'}, inplace=True)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
continuous_columns = df.select_dtypes(include=['Int64', 'float', 'int']).columns


def metricas (df, columna):
    
    if columna in categorical_columns:
        
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        sns.countplot(data=df, x=columna)
        plt.title(f'Distribución de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df, x=columna, y='precio_contado')
        plt.title(f'Boxplot de {columna} con precio')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()
    
    elif columna in continuous_columns:

        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 3, 1)
        sns.histplot(df[columna], kde=True)
        plt.title(f'Histograma de {columna}')
        
        
        plt.subplot(1, 3, 2)
        sns.boxplot(x=df[columna])
        plt.title(f'Boxplot de {columna}')
        plt.xticks(rotation=45, ha='right')
        
        plt.subplot(1, 3, 3)
        sns.scatterplot(x=df[columna], y=df['precio_contado'])
        plt.title(f'Scatterplot de {columna} con precio')
        plt.tight_layout()
        plt.xticks(rotation=45, ha='right')
        plt.show()

In [ ]:
categorical_columns

In [ ]:
len(categorical_columns)

In [ ]:
continuous_columns

In [ ]:
len(continuous_columns)

In [ ]:
for columna in continuous_columns:
    metricas(df, columna)

In [ ]:
for columna in categorical_columns:
    metricas(df, columna)

In [ ]:
continuous_columns

In [ ]:
correlation_matrix = df[continuous_columns].corr()

plt.figure(figsize=(25, 15))

sns.heatmap(correlation_matrix, annot=True)

plt.title('Mapa de Calor de Correlaciones')
plt.show()

In [ ]:
df.head()

In [ ]:
listado_provincias = {}
for index,detalle in enumerate (df['id_provincia'].unique(), start = 1):
    detalle = detalle
    
    listado_provincias[index] = detalle

df_provincias = pd.DataFrame.from_dict(listado_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_provincias.to_csv('../bin/listado_provincias.csv', index=False)

conversion = {v: k for k, v in listado_provincias.items()}
df['id_provincia'] = df['id_provincia'].map(conversion)

In [ ]:
listado_concesionarios = {}
for index,detalle in enumerate (df['id_concesionario'].unique(), start = 1):
    
    id_provincia = df[df['id_concesionario'] == detalle]['id_provincia'].iloc[0]
    
    listado_concesionarios[index] = {'nombre_concesionario': detalle, 'id_provincia': id_provincia}

df_concesionarios = pd.DataFrame.from_dict(listado_concesionarios, orient='index', columns=['nombre_concesionario', 'id_provincia']).reset_index()
df_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)
df_concesionarios.to_csv('../bin/listado_concesionarios.csv', index=False)

conversion = df_concesionarios.set_index('nombre_concesionario')['id_concesionario'].to_dict()

df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [ ]:
df_concesionarios

In [ ]:
listado_modelos = {}
for index,detalle in enumerate (df['id_modelo'].unique(), start = 1):
    detalle = detalle
    
    listado_modelos[index] = detalle

df_modelos = pd.DataFrame.from_dict(listado_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)
df_modelos.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_modelos.items()}
df['id_modelo'] = df['id_modelo'].map(conversion)

In [ ]:
listado_marcas = {}
for index,detalle in enumerate (df['id_marca'].unique(), start = 1):
    detalle = detalle
    
    listado_marcas[index] = detalle

df_marcas = pd.DataFrame.from_dict(listado_marcas, orient='index', columns=['nombre_marca']).reset_index()
df_marcas.rename(columns={'index': 'id_marca'}, inplace=True)
df_marcas.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_marcas.items()}
df['id_marca'] = df['id_marca'].map(conversion)

In [ ]:
listado_carrocerias = {}
for index,detalle in enumerate (df['id_carroceria'].unique(), start = 1):
    detalle = detalle
    
    listado_carrocerias[index] = detalle
    
df_carrocerias = pd.DataFrame.from_dict(listado_carrocerias, orient='index', columns=['nombre_carroceria']).reset_index()
df_carrocerias.rename(columns={'index': 'id_carroceria'}, inplace=True)
df_carrocerias.to_csv('../bin/listado_carrocerias.csv', index=False)

conversion = {v: k for k, v in listado_carrocerias.items()}
df['id_carroceria'] = df['id_carroceria'].map(conversion)

In [ ]:
listado_distintivo_ambiental = {}
for index,detalle in enumerate (df['id_distintivo_ambiental'].unique(), start = 1):
    detalle = detalle
    
    listado_distintivo_ambiental[index] = detalle
    
df_distintivo_ambiental = pd.DataFrame.from_dict(listado_distintivo_ambiental, orient='index', columns=['nombre_distintivo']).reset_index()
df_distintivo_ambiental.rename(columns={'index': 'id_distintivo'}, inplace=True)
df_distintivo_ambiental.to_csv('../bin/listado_distintivo_ambiental.csv', index=False)

conversion = {v: k for k, v in listado_distintivo_ambiental.items()}
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [ ]:
listado_combustibles = {}
for index,detalle in enumerate (df['combustible'].unique(), start = 1):
    detalle = detalle
    
    listado_combustibles[index] = detalle
    
df_combustibles = pd.DataFrame.from_dict(listado_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)
df_combustibles.to_csv('../bin/listado_combustibles.csv', index=False)

conversion = {v: k for k, v in listado_combustibles.items()}
df['combustible'] = df['combustible'].map(conversion)

In [ ]:
listado_tipos_cambio = {}
for index,detalle in enumerate (df['tipo_cambio'].unique(), start = 1):
    detalle = detalle
    
    listado_tipos_cambio[index] = detalle
    
df_tipos_cambio = pd.DataFrame.from_dict(listado_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)
df_tipos_cambio.to_csv('../bin/listado_tipos_cambio.csv', index=False)

tipos_cambio_to_indices = {v: k for k, v in listado_tipos_cambio.items()}
df['id_tipo_cambio'] = df['tipo_cambio'].map(tipos_cambio_to_indices)

df = df.drop(columns=['tipo_cambio'])

In [ ]:
listado_sobrealimentacion = {}
for index,detalle in enumerate (df['id_sobrealimentacion'].unique(), start = 1):
    detalle = detalle
    
    listado_sobrealimentacion[index] = detalle
    
df_sobrealimentacion = pd.DataFrame.from_dict(listado_sobrealimentacion, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_sobrealimentacion.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)
df_sobrealimentacion.to_csv('../bin/listado_sobrealimentacion.csv', index=False)

conversion = {v: k for k, v in listado_sobrealimentacion.items()}
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [ ]:
df = df.drop(columns=['traccion'])

In [ ]:
df.to_csv('../bin/prestaciones.csv', index=False)

In [ ]:
df.head()